In [1]:
import pandas as pd
from pandas_datareader import data
import yfinance as yf
yf.pdr_override()




In [14]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

def macd_diff(df,i):
    return df.loc[i,'macd']-df.loc[i,'signal']
    
def get_almost_macd_signal(df,hold,i):
    if (i<2):
        return False
    if (hold):
        if (macd_diff(df,i)<macd_diff(df,i-1) and macd_diff(df,i-1)<macd_diff(df,i-2) and macd_diff(df,i)>0 and macd_diff(df,i)<0.01):
            return True
        return False
    else:
        if (macd_diff(df,i)>macd_diff(df,i-1) and macd_diff(df,i-1)>macd_diff(df,i-2) and macd_diff(df,i)<0 and macd_diff(df,i)>-0.01):
            return True
        return False

def get_almost_macd_signal1(df,hold,i):
    if (i<2):
        return False
    if (hold):
        if (macd_diff(df,i)<macd_diff(df,i-1) and macd_diff(df,i-1)<macd_diff(df,i-2) and macd_diff(df,i)>0 and macd_diff(df,i)<2):
            return True
        return False
    else:
        if (macd_diff(df,i)>macd_diff(df,i-1) and macd_diff(df,i-1)>macd_diff(df,i-2) and macd_diff(df,i)<0 and macd_diff(df,i)>-2):
            return True
        return False
    
def get_almost_macd_signal2(df,hold,i):
    if (i<2):
        return False
    if (hold):
        if (macd_diff(df,i)<macd_diff(df,i-1) and macd_diff(df,i-1)<macd_diff(df,i-2) and macd_diff(df,i)>0 and macd_diff(df,i)<5):
            return True
        return False
    else:
        if (macd_diff(df,i)>macd_diff(df,i-1) and macd_diff(df,i-1)>macd_diff(df,i-2) and macd_diff(df,i)<0 and macd_diff(df,i)>-5):
            return True
        return False
    
    
    
    
def get_macd_signal(df,hold,i):
    if (hold):
        if ((df.iloc[i])['macd']<(df.iloc[i])['signal']):
            return True
        return False
    else:
        if ((df.iloc[i])['macd']>(df.iloc[i])['signal']):
            return True
        return False

def get_macd_signal_new(df,hold,i):
    if (i<1):
        return False
    
    if (hold):
        if (((df.iloc[i-1])['macd']>(df.iloc[i-1])['signal']) and ((df.iloc[i])['macd']<(df.iloc[i])['signal'])):
            return True
        return False
    else:
        if (((df.iloc[i-1])['macd']<(df.iloc[i-1])['signal']) and ((df.iloc[i])['macd']>(df.iloc[i])['signal'])):
            return True
        return False    
     
def naive_backtest(df,get_signal):
    pv=1000
    hold=False
    start_price=-float("inf")
    
    total_trade=0
    win_count=0
    
    #count=0
    for i in range(len(df)):
        if (hold):
            if (get_signal(df,hold,i)):
                end_price=df.iloc[i]['Adj Close']
                pv*=end_price/start_price
                hold=False
                
                total_trade+=1
                if (end_price>start_price):
                    win_count+=1
        else:
            if (get_signal(df,hold,i)):
                #count+=1
                start_price=df.iloc[i]['Adj Close']
                hold=True
    #print(count) 
    return (pv,total_trade,win_count)


def backtest_one_day(df,get_signal):
    pv=1000

    total_trade=0
    win_count=0
    
    #count=0
    for i in range(len(df)-1):
        if (get_signal(df,False,i)):
            #count+=1
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=end_price/start_price

            total_trade+=1
            if (end_price>start_price):
                win_count+=1
            #print(pv,start_price,end_price)
    #print(count)
    return (pv,total_trade,win_count)

def backtest_one_day_short_possible(df,get_signal):
    pv=1000

    total_trade=0
    win_count=0
    
    for i in range(len(df)-1):
        if (get_signal(df,False,i)):
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=end_price/start_price
            
            total_trade+=1
            if (end_price>start_price):
                win_count+=1
        elif (get_signal(df,True,i)):
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=start_price/end_price
            
            total_trade+=1
            if (end_price<start_price):
                win_count+=1
        
    return (pv,total_trade,win_count)

China_A_list=['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz','301487.sz','300570.sz'\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']
#200 stocks randomly sampled from all stocks in NASDAQ
#in the time range 2010-01-01 to 2023-08-01, only 183 stocks are available
nasdaq_100=['FI','AAPL', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'ASML', 'AVGO', 'PEP', 'COST', 'AZN', 'CSCO', 'TMUS', 'ADBE', 'TXN', 'AMD', 'CMCSA', 'NFLX', 'QCOM', 'HON', 'AMGN', 'INTU', 'INTC', 'SBUX', 'AMAT', 'GILD', 'PDD', 'ADI', 'BKNG', 'MDLZ', 'ADP', 'REGN', 'ISRG', 'PYPL', 'VRTX', 'ABNB', 'LRCX', 'MU', 'ATVI', 'MELI', 'PANW', 'CSX', 'CHTR', 'SNPS', 'MRNA', 'CDNS', 'JD', 'KLAC', 'MNST', 'FTNT', 'ORLY', 'MAR', 'KDP', 'NXPI', 'KHC', 'MCHP', 'DXCM', 'CTAS', 'AEP', 'ADSK', 'IDXX', 'EXC', 'PAYX', 'BIIB', 'WDAY', 'LULU', 'SGEN', 'PCAR', 'ODFL', 'MRVL', 'WBD', 'ROST', 'XEL', 'ILMN', 'CPRT', 'EA', 'CRWD', 'DLTR', 'CTSH', 'FAST', 'VRSK', 'ENPH', 'WBA', 'ANSS', 'BKR', 'CSGP', 'CEG', 'ALGN', 'FANG', 'TEAM', 'EBAY', 'DDOG', 'ZM', 'ZS', 'LCID', 'SIRI', 'RIVN']
nasdaq_random=['BOXL', 'PBLA', 'NMIH', 'OCCI', 'ONYXU', 'BRFH', 'ATRC', 'VISL', 'AMBA', 'ANDE', 'BIOL', 'MASI', 'FNCB', 'GDYN', 'RDUS', 'SGMA', 'BIAF', 'OSUR', 'DCPH', 'CPRT', 'AFBI', 'BLACU', 'AKU', 'TBMC', 'ME', 'CLPT', 'HURC', 'CHKEW', 'COOLW', 'GENK', 'BTBT', 'GLUE', 'CNOBP', 'APGE', 'UCTT', 'VERU', 'CSWCZ', 'PEGY', 'HTZ', 'IRTC', 'BMBL', 'PLXS', 'LINK', 'ZEUS', 'HOWL', 'CVKD', 'UG', 'XRX', 'ELVN', 'IOVA', 'INBX', 'VIAV', 'BBIO', 'LUXH', 'FRBA', 'COHU', 'PUBM', 'INDB', 'AMAT', 'FEIM', 'ATER', 'ADBE', 'FINW', 'BYRN', 'AVT', 'MDWT', 'SVC', 'IDN', 'ERII', 'DMRC', 'CGEM', 'CPHC', 'OCUP', 'SURF', 'VTNR', 'TFSL', 'BSET', 'IVCP', 'WAVD', 'VMD', 'EAR', 'TILE', 'GLLI', 'VTRS', 'GNLN', 'HGBL', 'MCAF', 'SONN', 'CAPR', 'PGC', 'XOMAO', 'VALU', 'ACVA', 'LIVE', 'GOGO', 'EFSC', 'APDN', 'MXL', 'IDCC', 'WINT', 'WERN', 'DKNG', 'AMPGW', 'LQDT', 'BCAB', 'CNFR', 'ALKT', 'POWL', 'SRDX', 'DUOT', 'HFWA', 'MPRA', 'CZFS', 'CDXS', 'CGBD', 'GLSI', 'OXSQ', 'CRVS', 'MSSAU', 'BTCY', 'LFMD', 'MTEX', 'TECTP', 'PAYS', 'KOD', 'PROV', 'BSFC', 'PFTA', 'JNVR', 'ZI', 'FMAO', 'CARA', 'VRA', 'RILYK', 'INBK', 'ENG', 'TRINL', 'PRSR', 'THRM', 'PTPI', 'IMMX', 'ISPO', 'BVS', 'ARCC', 'APPN', 'BLKB', 'OXSQG', 'PATK', 'KZR', 'GH', 'GIA', 'ADEA', 'ICUI', 'BDTX', 'ISPR', 'ADMA', 'INBKZ', 'UONEK', 'UTRS', 'ACAX']

In [93]:
#almost MACD: how often does it get an MACD golden crossover?
temp=[]

import time

counts={}

#start=time.time()

for span in range(3,10):
    total_count=0
    MACD_count=0
    for t in nasdaq_random:
        df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01',progress=False)
        df_macd = get_macd(df['Close'], 26, 12, span)
        df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
        for i in range(2,len(df_enlarged)-1):
            if (macd_diff(df_enlarged,i)>macd_diff(df_enlarged,i-1) and \
                macd_diff(df_enlarged,i-1)>macd_diff(df_enlarged,i-2) and macd_diff(df_enlarged,i)<0 \
                and macd_diff(df_enlarged,i)>-0.01):
                total_count+=1
                if (macd_diff(df_enlarged,i+1)>0 and macd_diff(df_enlarged,i)<0):
                    MACD_count+=1
                
    counts[span]=(MACD_count,total_count)

#end=time.time()    
#print(end-start)
    
temp=[]
for span in range(3,10):
    temp.append(list(counts[span]))

res=[]
for _ in range(len(temp)):
    tmp=[0]*len(temp[0])
    res.append(tmp)
df_plot=pd.DataFrame(res)

for i in range(len(temp)):
    for j in range(len(temp[0])):
        df_plot.loc[i,j]=temp[i][j]
        
df_plot.to_excel("almost_MACD_find_MACD.xlsx")

In [92]:
#find how many times there will be earning when an MACD golden crossover appears.
temp=[]

import time

counts={}

#start=time.time()
res={}
long_c={}
total_c={}
for span in range(3,10):
    total_count=0
    long_count=0
    
    
    res[span]=[]
    long_c[span]={}
    total_c[span]={}
    
    maxres=-float("inf")
    minres=float("inf")
    for t in ['CSWCZ']:
        
        long_c[span][t]=0
        total_c[span][t]=0
        
        total_profit=1
        df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01',progress=False)
        df_macd = get_macd(df['Close'], 26, 12, span)
        df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
        for i in range(1,len(df_enlarged)):
            if (macd_diff(df_enlarged,i)>0 and macd_diff(df_enlarged,i-1)<0):
                total_c[span][t]+=1
                total_count+=1
                if (df_enlarged.loc[i,'Adj Close']>df_enlarged.loc[i-1,'Adj Close']):
                    long_c[span][t]+=1
                    long_count+=1
                total_profit*=(df_enlarged.loc[i,'Adj Close']/df_enlarged.loc[i-1,'Adj Close'])
                #print(span,total_profit,df_enlarged.loc[i,'Adj Close']/df_enlarged.loc[i-1,'Adj Close'],total_count)
             
                #total_profit+=(df_enlarged.loc[i,'Close']-df_enlarged.loc[i-1,'Close'])
                #print(df_enlarged.loc[i,'Close']-df_enlarged.loc[i-1,'Close'])
        minres=min(total_profit,minres)
        maxres=max(total_profit,maxres)
        res[span].append(total_profit)
        
    counts[span]=(long_count,total_count,minres,maxres)

#end=time.time()    
#print(end-start)
    
temp=[]
for span in range(3,10):
    temp.append(list(counts[span]))

rs=[]
for _ in range(len(temp)):
    tmp=[0]*len(temp[0])
    rs.append(tmp)
df_plot=pd.DataFrame(rs)

for i in range(len(temp)):
    for j in range(len(temp[0])):
        df_plot.loc[i,j]=temp[i][j]
        
#df_plot.to_excel("long_on_the_day_of_crossover.xlsx")

#import pandas as pd
#pd.DataFrame.from_dict(res, orient='index',columns=nasdaq_random).to_excel("sell_on_MACD_crossover_nasdaq_random.xlsx")

3 1.001980167804378 1.001980167804378 1
3 1.0048352840629287 1.0028494738222284 2
4 1.001980167804378 1.001980167804378 1
4 1.0048352840629287 1.0028494738222284 2
5 1.001980167804378 1.001980167804378 1
5 1.0048352840629287 1.0028494738222284 2
6 1.0028494738222284 1.0028494738222284 1
7 1.0028494738222284 1.0028494738222284 1
8 1.0028494738222284 1.0028494738222284 1
9 1.0028494738222284 1.0028494738222284 1


In [25]:
#对应“MACD金死叉对单调2”表格
temp=[]
for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz','301487.sz','300570.sz'\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']:
    df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
    df_macd = get_macd(df['Close'], 26, 12, 9)
    df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
    
    temp.append([])
    a=naive_backtest(df_enlarged,get_macd_signal)
    print(a,end="")
    temp[-1].extend(a)
    
    a=naive_backtest(df_enlarged,get_almost_macd_signal)
    print(a)
    temp[-1].extend(a)

[*********************100%***********************]  1 of 1 completed
(3030.4992294569815, 131, 51)(4757.280510909064, 12, 9)
[*********************100%***********************]  1 of 1 completed
(5020.7771970363165, 58, 23)(1427.7342485350446, 9, 4)
[*********************100%***********************]  1 of 1 completed
(3601.876752764861, 133, 48)(6293.906175466592, 38, 18)
[*********************100%***********************]  1 of 1 completed
(1131.7115042397054, 65, 18)(1639.9734517979168, 2, 2)
[*********************100%***********************]  1 of 1 completed
(4297.386825987679, 136, 56)(1498.5760016915358, 1, 1)
[*********************100%***********************]  1 of 1 completed
(6042.21750203685, 109, 43)(7219.057799737428, 26, 15)
[*********************100%***********************]  1 of 1 completed
(2532.1727838476286, 124, 47)(1200.9537913625247, 28, 15)
[*********************100%***********************]  1 of 1 completed
(2282.607014746203, 100, 30)(1424.9263718007655, 42, 16)
[


1 Failed download:
['301487.SZ']: Exception("%ticker%: Data doesn't exist for startDate = 1262275200, endDate = 1690819200")



(1000, 0, 0)(1000, 0, 0)
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['300570.SZ688359.SS']: Exception('%ticker%: No timezone found, symbol may be delisted')



(1000, 0, 0)(1000, 0, 0)
[*********************100%***********************]  1 of 1 completed
(961.297950432339, 2, 0)(1000, 0, 0)
[*********************100%***********************]  1 of 1 completed
(1656.3364376157372, 107, 31)(1924.588105185878, 50, 22)
[*********************100%***********************]  1 of 1 completed
(1249.3561691435189, 123, 41)(847.7195228631375, 16, 7)
[*********************100%***********************]  1 of 1 completed
(2593.4712563442536, 38, 19)(1000, 0, 0)
[*********************100%***********************]  1 of 1 completed
(11881.610966892324, 125, 47)(1559.3524486518288, 15, 9)
[*********************100%***********************]  1 of 1 completed
(3176.784719098874, 118, 49)(2900.40806138222, 23, 13)
[*********************100%***********************]  1 of 1 completed
(938.2106812875599, 140, 52)(2512.782472028601, 39, 23)
[*********************100%***********************]  1 of 1 completed
(2044.5911827075674, 126, 48)(2062.3828839160537, 23, 15)
[****

In [95]:
df_plot.to_excel(f"MACD_and_almost_MACD{span}.xlsx")

In [103]:
#check how the strategy of buying on the day of MACD golden crossover and buying when almost_MACD is detected
#behave w.r.t. each other

for span in range(3,10):
    temp=[]
    for t in nasdaq_random:
        #start=time.time()
        
        df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01',progress=False)
        df_macd = get_macd(df['Close'], 26, 12, span)
        df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()

        temp.append([])
        a=backtest_one_day(df_enlarged,get_macd_signal_new)
        #print(a,end="")
        temp[-1].extend(a)

        a=backtest_one_day(df_enlarged,get_almost_macd_signal)
        #print(a)
        temp[-1].extend(a)
 
        #end=time.time()
    
        
    
    res=[]
    for _ in range(len(temp)):
        tmp=[0]*len(temp[0])
        res.append(tmp)
    df_plot=pd.DataFrame(res)

    for i in range(len(temp)):
        for j in range(len(temp[0])):
            df_plot.loc[i,j]=temp[i][j]

    df_plot.to_excel(f"MACD_and_almost_MACD{span}.xlsx")
 

#print(end-start)

In [15]:

for span in range(3,10):
    temp=[]
    for t in nasdaq_random:
        #start=time.time()
        
        df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01',progress=False)
        df_macd = get_macd(df['Close'], 26, 12, span)
        df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
 
        temp.append([])
        a=backtest_one_day(df_enlarged,get_almost_macd_signal1)
        #print(a,end="")
        temp[-1].extend(a)

        a=backtest_one_day(df_enlarged,get_almost_macd_signal2)
        #print(a)
        temp[-1].extend(a)
 
        #end=time.time()
    res=[]
    for _ in range(len(temp)):
        tmp=[0]*len(temp[0])
        res.append(tmp)
    df_plot=pd.DataFrame(res)

    for i in range(len(temp)):
        for j in range(len(temp[0])):
            df_plot.loc[i,j]=temp[i][j]

    df_plot.to_excel(f"MACD_and_almost_MACD{span}.xlsx")
 
        
    


In [ ]:
temp=[]
for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz',\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']:
    df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
    df_macd = get_macd(df['Close'], 26, 12, 9)
    df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
    
    temp.append([])
    #a=backtest_one_day_short_possible(df_enlarged,get_macd_signal_new)
    #print(a,end="")
    #temp[-1].extend(a)
    
    a=backtest_one_day_short_possible(df_enlarged,get_almost_macd_signal)
    print(a)
    temp[-1].extend(a)

In [ ]:
temp=[]
for span in range(3,10):
    temp.append(list(counts[span]))

res=[]
for _ in range(len(temp)):
    tmp=[0]*len(temp[0])
    res.append(tmp)
df_plot=pd.DataFrame(res)

for i in range(len(temp)):
    for j in range(len(temp[0])):
        df_plot.loc[i,j]=temp[i][j]
        
df_plot.to_excel("long_on_the_day_of_crossover.xlsx")